In [1]:
# Create a function for easily querying data.
import pymysql
import pandas as pd
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
import plotly.express as px

def qry(sql):

    # Open database connection
    connection = pymysql.connect(host="34.133.40.159",user="root",password="rootroot",database="project" )
    df = pd.read_sql(sql, connection)

    # disconnect from server
    connection.close()
    
    # return data.
    return df

In [2]:
total = qry("""

SELECT 
	brand_name,
    COUNT(sentiment_word) as total_words
FROM
	brands,
	tweets,
    sentimental_data
WHERE 
	tweets.tweet_id=sentimental_data.tweet_id AND
    brands.brand_index=tweets.brand_index
GROUP BY 
	brand_name
ORDER BY
	total_words DESC


""")

In [3]:
pos = qry("""

SELECT 
	brand_name,
    COUNT(sentiment_word) as pos_words
FROM
	brands,
	tweets,
    sentimental_data
WHERE 
	tweets.tweet_id=sentimental_data.tweet_id AND
    brands.brand_index=tweets.brand_index AND
    sentiment_word = 'positive'
GROUP BY 
	brand_name
ORDER BY
	pos_words DESC


""")

In [4]:
neg = qry("""

SELECT 
	brand_name,
    COUNT(sentiment_word) as neg_words
FROM
	brands,
	tweets,
    sentimental_data
WHERE 
	tweets.tweet_id=sentimental_data.tweet_id AND
    brands.brand_index=tweets.brand_index AND
    sentiment_word = 'negative'
GROUP BY 
	brand_name
ORDER BY
	neg_words DESC 


""")

In [5]:
neu = qry("""

SELECT 
	brand_name,
    COUNT(sentiment_word) as neu_words
FROM
	brands,
	tweets,
    sentimental_data
WHERE 
	tweets.tweet_id=sentimental_data.tweet_id AND
    brands.brand_index=tweets.brand_index AND
    sentiment_word = 'neutral'
GROUP BY 
	brand_name
ORDER BY
	neu_words DESC


""")


In [6]:
merged = total.merge(pos, left_on = "brand_name", right_on = "brand_name").merge(neg, left_on = "brand_name", right_on = "brand_name").merge(neu, left_on = 'brand_name', right_on = 'brand_name')

In [7]:
merged['pos_pct'] = merged['pos_words'] / merged['total_words']
merged['neg_pct'] = merged['neg_words'] / merged['total_words']
merged['neu_pct'] = merged['neu_words'] / merged['total_words']

merged=merged.sort_values('total_words', ascending=False).head(10)
merged=merged.drop(['total_words','pos_words','neg_words','neu_words'], axis = 1)

In [8]:
merged.set_index('brand_name', inplace=True)

In [17]:
fig = px.bar(merged, y=['pos_pct','neu_pct','neg_pct'],  title="Plot Title")
fig.update_layout(title="Brand Perception Based On Sentimental Analysis", yaxis_title="Proportion")

In [10]:
fig = px.bar(df, x="brand_name", y="total_words", barmode="group")
fig.show()


NameError: name 'df' is not defined